# Generate the Individual Models

Here, the individual models are created. These models use 80% of a subject's data for training and 20% for testing. They are called individual models because each model only uses data from a single subject.

There are 14 different subjects, and 18 models are generated for each subject. The total number of models generated is 14 × 18 = 252.

The models are generated and then saved in a CSV file called `train_individual.csv` for the training models and `test_individual.csv` for the testing models. The feature importance data is also saved for each model. This data is saved in a CSV file called `feature_importance_individual.csv`. These files are in the directory `outputs/individual/`.

In [1]:
from pycaret.datasets import get_data
from pycaret.regression import *
import csv
import pandas as pd

In [2]:
df = get_data('data/initial_features_mod')

,subject,experimental_condition,rpe,wrist_acc_time,wrist_acc_length,wrist_acc_mean,wrist_acc_rms,wrist_acc_mad,wrist_acc_std,wrist_acc_min,...,upperarm_jerk_rms,upperarm_jerk_mad,upperarm_jerk_std,upperarm_jerk_min,upperarm_jerk_max,upperarm_jerk_med,upperarm_jerk_perc25,upperarm_jerk_perc75,weight,pace
0,1,Condition 2.5-15,0,1.631984e+09,2952,1.017365,1.042456,0.153341,0.227376,0.426780,...,4.277703,2.170631,2.824283,0.014541,19.016887,2.537380,0.980603,4.603277,2.5,15
1,1,Condition 2.5-15,0,1.631984e+09,2414,1.019116,1.046801,0.166215,0.239201,0.481607,...,4.663319,2.295026,3.278950,0.036583,31.471509,2.540472,1.007865,4.469952,2.5,15
2,1,Condition 2.5-15,0,1.631984e+09,2374,1.020968,1.053935,0.183874,0.261596,0.478265,...,4.684816,2.225496,3.102505,0.027138,29.295564,2.826385,1.357187,4.803642,2.5,15
3,1,Condition 2.5-15,0,1.631984e+09,2455,1.015247,1.039529,0.167656,0.223415,0.452296,...,3.675835,1.761217,2.281765,0.024198,18.058972,2.464632,1.135283,4.091388,2.5,15
4,1,Condition 2.5-15,0,1.631984e+09,2502,1.022933,1.051561,0.167432,0.243747,0.396358,...,4.896867,2.419522,3.281058,0.029791,27.380172,2.791626,1.318934,4.934743,2.5,15


In [4]:
with open('outputs/individual/train_individual.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "MAE", "MSE", "RMSE", "R2", "RMSLE", "MAPE", "TT (Sec)", "Subject"])

with open('outputs/individual/test_individual.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Model", "MAE", "MSE", "RMSE", "R2", "RMSLE", "MAPE", "Subject"])

with open('outputs/individual/feature_importance_individual.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Feature", "Value", "Model", "Subject"])

# Predictors to remove: experimental_condition, subject
for subject in df['subject'].unique():
    # Splits the data 80 - 20
    train = df[df['subject'] == subject].sample(frac=0.8, random_state=42)
    test = df.drop(train.index)
    
    reg = setup(data=train, target='rpe', ignore_features=['experimental_condition', 'subject'])
    best = compare_models(sort='MAE', n_select = 18)
    all = pull()
    all['subject'] = str(subject)

    all.to_csv('outputs/individual/train_individual.csv', mode='a', header=False, index=False)
    
    test_results = pd.DataFrame()
    for model in best:
        test_result = predict_model(model, verbose = False)
        test_result = pull()
        test_results = test_results.append(test_result)
        
        # NEW: MAKE SURE IT WORKS
        try:
            importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.feature_importances_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
        except:
            try:
                importance = pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(model.coef_)}).sort_values(by='Value', ascending=False).reset_index().drop('index', axis=1)
            except:
                importance = pd.DataFrame({'Feature': ['error'], 'Value': [0]})
        importance['Model'] = str(model)
        importance['Subject'] = str(subject)
        importance.to_csv('outputs/individual/feature_importance_individual.csv', mode='a', header=False, index=False)
        

    test_results['subject'] = str(subject)
    test_results.to_csv('outputs/individual/test_individual.csv', mode='a', header=False, index=False)

,Description,Value
0,Session id,7337
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(224, 61)"
4,Transformed data shape,"(224, 59)"
5,Transformed train set shape,"(156, 59)"
6,Transformed test set shape,"(68, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.1260,0.0566,0.2274,0.9409,0.1358,0.1151,0.0520
dt,Decision Tree Regressor,0.2246,0.3012,0.5128,0.6803,0.3062,0.1033,0.0500
gbr,Gradient Boosting Regressor,0.2496,0.1474,0.3739,0.8452,0.2166,0.1427,0.1480
et,Extra Trees Regressor,0.2825,0.1624,0.3999,0.8236,0.2466,0.1531,0.1440
rf,Random Forest Regressor,0.3352,0.1987,0.4426,0.7867,0.2575,0.1731,0.1720
ada,AdaBoost Regressor,0.3518,0.2093,0.4514,0.7736,0.2745,0.2066,0.1130
lightgbm,Light Gradient Boosting Machine,0.3659,0.2328,0.4771,0.7555,0.2731,0.2443,0.0550
ridge,Ridge Regression,0.6206,0.5952,0.7641,0.3645,0.4049,0.3803,0.0520
br,Bayesian Ridge,0.6590,0.6759,0.8126,0.2872,0.4124,0.3590,0.0510
llar,Lasso Least Angle Regression,0.6635,0.6774,0.8143,0.2846,0.4139,0.3652,0.0530


,Description,Value
0,Session id,1417
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(248, 61)"
4,Transformed data shape,"(248, 59)"
5,Transformed train set shape,"(173, 59)"
6,Transformed test set shape,"(75, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.2873,0.1879,0.4244,0.9621,0.1855,0.0910,0.0530
gbr,Gradient Boosting Regressor,0.6181,0.7698,0.8460,0.8467,0.3171,0.1853,0.1560
rf,Random Forest Regressor,0.7221,1.0721,1.0032,0.7944,0.3517,0.1745,0.1880
dt,Decision Tree Regressor,0.7255,2.3222,1.3863,0.5376,0.4448,0.2357,0.0550
ada,AdaBoost Regressor,0.7564,0.9915,0.9723,0.8024,0.3666,0.2113,0.1090
lightgbm,Light Gradient Boosting Machine,0.7933,1.1151,1.0340,0.7782,0.3597,0.2274,0.0640
et,Extra Trees Regressor,0.8775,1.2866,1.1219,0.7500,0.3788,0.2289,0.1500
ridge,Ridge Regression,1.4355,3.5182,1.8599,0.3042,0.5297,0.3837,0.0540
br,Bayesian Ridge,1.4913,3.7034,1.9100,0.2692,0.5447,0.3998,0.0570
lr,Linear Regression,1.6282,4.2087,2.0246,0.1788,0.5661,0.5382,0.0550


,Description,Value
0,Session id,2068
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(157, 61)"
4,Transformed data shape,"(157, 59)"
5,Transformed train set shape,"(109, 59)"
6,Transformed test set shape,"(48, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.3621,0.3200,0.5394,0.9438,0.1754,0.1400,0.0810
gbr,Gradient Boosting Regressor,0.7964,1.2657,1.0963,0.7708,0.3447,0.3266,0.1560
rf,Random Forest Regressor,0.8528,1.3878,1.1452,0.7549,0.3642,0.3465,0.2010
lightgbm,Light Gradient Boosting Machine,0.9182,1.6985,1.2632,0.6968,0.3901,0.3873,0.0870
ada,AdaBoost Regressor,0.9334,1.5979,1.2335,0.7066,0.4254,0.3564,0.1220
dt,Decision Tree Regressor,0.9767,2.1225,1.4244,0.5905,0.4549,0.4072,0.0780
et,Extra Trees Regressor,1.1273,2.0908,1.4137,0.6165,0.4474,0.4402,0.1600
lr,Linear Regression,1.5587,4.1629,1.9936,0.2373,0.6112,0.6642,0.0810
ridge,Ridge Regression,1.6910,4.0162,1.9744,0.2748,0.6187,0.6034,0.0820
omp,Orthogonal Matching Pursuit,1.9149,5.1492,2.2360,0.0939,0.6980,0.6525,0.0770


,Description,Value
0,Session id,4011
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(97, 61)"
4,Transformed data shape,"(97, 59)"
5,Transformed train set shape,"(67, 59)"
6,Transformed test set shape,"(30, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.7167,0.7620,0.8648,0.8347,0.3215,0.1801,0.0880
gbr,Gradient Boosting Regressor,1.7422,4.7090,2.0758,-0.0052,0.6158,0.4738,0.1360
rf,Random Forest Regressor,1.9412,5.5054,2.2987,-0.1582,0.6963,0.4620,0.1770
ada,AdaBoost Regressor,1.9419,5.8067,2.3659,-0.2750,0.6732,0.4903,0.1140
et,Extra Trees Regressor,1.9606,5.7582,2.3284,-0.2249,0.6875,0.4692,0.1590
lightgbm,Light Gradient Boosting Machine,2.0518,6.3088,2.4180,-0.1197,0.7001,0.4839,0.0880
dt,Decision Tree Regressor,2.0595,8.4071,2.8111,-0.4769,0.7661,0.5576,0.0870
ridge,Ridge Regression,2.4212,8.7220,2.7111,-0.4780,0.7615,0.6901,0.0880
dummy,Dummy Regressor,2.5082,8.4856,2.8682,-0.5318,0.8189,0.5844,0.0930
huber,Huber Regressor,2.5357,8.9055,2.9364,-0.5807,0.8239,0.5881,0.0930


,Description,Value
0,Session id,8855
1,Target,rpe
2,Target type,Regression
3,Original data shape,"(170, 61)"
4,Transformed data shape,"(170, 59)"
5,Transformed train set shape,"(118, 59)"
6,Transformed test set shape,"(52, 59)"
7,Ignore features,2
8,Numeric features,58
9,Preprocess,True


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
knn,K Neighbors Regressor,0.2636,0.1265,0.3461,0.9512,0.1485,0.1108,0.0890
dt,Decision Tree Regressor,0.4674,0.5705,0.7423,0.7532,0.2956,0.2250,0.0880
gbr,Gradient Boosting Regressor,0.4903,0.4881,0.6671,0.8147,0.2429,0.1991,0.1410
rf,Random Forest Regressor,0.5559,0.6030,0.7473,0.7748,0.2749,0.2012,0.1850
lr,Linear Regression,0.5643,0.5773,0.7329,0.7862,0.2585,0.2998,0.0900
ada,AdaBoost Regressor,0.5757,0.5825,0.7359,0.7876,0.2936,0.2279,0.1280
lightgbm,Light Gradient Boosting Machine,0.6429,0.6735,0.8033,0.7345,0.3157,0.2640,0.0960
et,Extra Trees Regressor,0.6862,0.9146,0.9121,0.6480,0.3294,0.2423,0.1540
ridge,Ridge Regression,0.8798,1.3254,1.1268,0.4906,0.4100,0.3709,0.0890
br,Bayesian Ridge,0.9096,1.4141,1.1613,0.4694,0.4141,0.3653,0.0890
